# Nobel Prize analysis in SQL

## Import of data, libraries and making it ready to work with SQL

In [ ]:
# import librariesa
import pandas as pd
import sqlite3

In [ ]:
# install the ipython-sql libray
!pip install ipython-sql 

In [ ]:
# read the csv file to pandas dataframe
df = pd.read_csv('nobel.csv')

In [ ]:
# We will use sqlite3 library and create a connection
cnn = sqlite3.connect('jupyter_sql_nobel.db')
df.to_sql('nobel', cnn, if_exists='replace')
%load_ext sql
%sql sqlite:///jupyter_sql_nobel.db

## Check of the data

In [ ]:
%%sql
/* check general information about the data*/
PRAGMA table_info(nobel);

In [ ]:
%%sql
/* check first row to get idea how the data looks */
SELECT *
FROM nobel
LIMIT 1;

In [ ]:
%%sql
/* We are going to use birth_date and death_date in many queries; 
therefore, it is important to check that values are valid 
and we can do calculations with them using date and time functions. 
SQLite does not have a storage class set aside for storing dates and/or times. 
Instead, the built-in Date And Time Functions of SQLite are capable of storing
dates and times as TEXT, REAL, or INTEGER values.
See here: https://www.sqlite.org/datatype3.html */

        
SELECT
laureate_id,
birth_date,
strftime('%Y',birth_date) AS birth_strf
FROM nobel
WHERE birth_date IS NOT NULL AND birth_strf IS NULL;

In [ ]:
%%sql
/* Same check as in previous cell but for the death date*/

        
SELECT
laureate_id,
death_date,
strftime('%Y',death_date) AS death_strf
FROM nobel
WHERE death_date IS NOT NULL AND death_strf IS NULL;

### Birth_date issue and its solution
We can see that death dates are not cousing problems but some birth dates has a format YYYY-00-00. This is not recognized by time and date functions (e.g.: strftime, julianday) as a date and returns NULL values instead of year if we use strftime('%Y',birth_date). Therefore, these birth dates cannot be used to calculate age.

To clean this, the problematic birth_date values has to be removed or the dates changed to valid date. I have decided for the latter. If we change YYYY-00-00 to YYYY-07-01 then the birth_date will be almost exactly in the middle of the year and there will be in the worst case half year error for each of the laureates with edited birth_date. However, this update of nobel table will be done later after we will check if there are some dates on which was born the most laureates.

### Checking of missing values

In [ ]:
%%sql
/* count total number of rows and number of "NOT NULL" values 
in each column to get overview how complete are the data */

SELECT
    COUNT(*) AS total_nr,
    COUNT(year) AS year,
    COUNT(category) AS category,
    COUNT(prize) AS prize,
    COUNT(motivation) AS motivation,
    COUNT(prize_share) AS share,
    COUNT(laureate_id) AS laureate_id,
    COUNT(laureate_type) AS laureate_type,
    COUNT(full_name) AS full_name,
    COUNT(birth_date) AS birth_date,
    COUNT(birth_city) AS birth_city,
    COUNT(birth_country) AS birth_country,
    COUNT(sex) AS sex,
    COUNT(organization_name) AS organization_name,
    COUNT(organization_city) AS organization_city,
    COUNT(organization_country) AS organization_country,
    COUNT(death_date) AS death_date,
    COUNT(death_city) AS death_city,
    COUNT(death_country) AS death_country
FROM nobel;

### Overview of missing values
We can see that currently (Fabruary 2024) was historically awarded 1000 Nobel Prizes. First column with significant amount of missing values is 'motivation'. Further, there are around 40 values missing in columns related to birth. Generally, it would not be surprising if the birth_date or place of birth would be unknow for few people. Most likely, where is missing birth_country there will be also missing information on birth_city, however, it should be checked if the same rows have also problem with missing birth_date. Next we can see that there is missing information on sex at 30 cases which can be due to the fact that some prizes were awarded to institutions. Then, there is significant amount of data missing in columns related to organization and it will be worth of finding out reasons for that. Finally, lots of values are missing in relation to deaths, probably a big portion of awarded people is still alive or the data are not up to date. However, it is not focus of this work to check if any death_date is missing incorrectly and we will just assume that missing death_date means that the person is still alive.

In [ ]:
%%sql
/* check columns with missing data to find out if there is any reason for that
or if they have something in common. Lets check motivation column first */

SELECT *
FROM nobel
WHERE motivation IS NULL;

In [ ]:
%%sql
/* It seems that motivation was not announced for The Nobel Peace Prizes until year 1990.
We can double check it by counting missing motivation within Peace category. Indeed, 88+912 = 1000*/
SELECT COUNT(*)
FROM nobel
WHERE motivation IS NULL AND category = 'Peace';

In [ ]:
%%sql
/* Was there any Nobel Peace Prize with motivation before 1990? */
SELECT *
FROM nobel
WHERE category = 'Peace' AND motivation IS NOT NULL AND year<1990;

In [ ]:
%%sql
/* check columns with missing data to find out if there is any reason for that
or if they have something in common. Lets check birth and sex related columns now */

SELECT COUNT(*)
FROM nobel
WHERE birth_date IS NULL
OR birth_city IS NULL
OR birth_country IS NULL
OR sex IS NULL;

In [ ]:
%%sql
/* From earlier we know that there was 30-40 missing values 
in each of the columns related to birth or sex. 
From the result of previous query we can conclude that most of these 
missing values are occuring on the same rows.
Lets see full information if there is anything in common */

SELECT *
FROM nobel
WHERE birth_date IS NULL
OR birth_city IS NULL
OR birth_country IS NULL
OR sex IS NULL;

In [ ]:
%%sql
/* Mostly, the laureate_type is organization. 
In addition, organizations seems to be missing values 
at organization_name, organization_city, and organization_country column.
Lets check if any organization has a birth_date. */

SELECT *
FROM nobel
WHERE laureate_type LIKE 'Org%' AND birth_date IS NOT NULL;

In [ ]:
%%sql
/* Now, we can look which individuals are missing birth_date */

SELECT *
FROM nobel
WHERE laureate_type LIKE 'Ind%' AND birth_date IS NULL;

In [ ]:
%%sql
/* check columns with missing data to find out if there is any reason for that
or if they have something in common. Finally, lets check columns related to 
organization_name, _city, and _country where was 735 records out of 1000. */

SELECT *
FROM nobel
WHERE organization_name IS NULL
OR organization_city IS NULL
OR organization_country IS NULL;

In [ ]:
%%sql
/* As noticed earlier, organizations are suprisingly missing information
in organization_name, _city, and _country columns. Otherwise it seems that the most missing
values are related to Nobel Prizes in category of Peace and Literature.
Lets check which laureates in these two categories have values at organization columns */

SELECT *
FROM nobel
WHERE (category = 'Peace' OR
category = 'Literature')
AND (organization_name IS NOT NULL
OR organization_city IS NOT NULL
OR organization_country IS NOT NULL);

In [ ]:
%%sql
/* Seems that almost all prizes in category of Peace and all in Literature 
were given to individuals or organizations who are not part of any organization. 
LEts briefly check how many prizes was given in each of the two categories 
to see if it is close to number of missing values (ca 265) in organization columns*/

SELECT COUNT(category) AS nr_prizes_in_peace_or_literature
FROM nobel
WHERE (category = 'Peace' OR
category = 'Literature');

### Summary on missing values
We found out that motivation of Nobel Prize was not being announced for almost first 90 years of this prize existence. 

Further, most of the missing values related to birth and sex are connected to awarded organizations and few individuals. While there are only two individuals with missing birth date, the organizations are missing birth dates almost always. However, there are four exceptional records which are assigned as organizations but are clearly related to known individuals such as Dalai Lama or Mother Teresa. Note that missing birth dates in the case of organizations will be beneficial in following analysis when we will calculate lenght of life because it cannot be calculated without birth date; therefore, we can be sure all results will be related to individuals and not to organizations.

Finally, values missing in organization columns (_name, _city, _country) belongs to cases where laureate_type is organization or cases where category is peace or literature. Indeed, this makes sense and indicates that organizations in organization columns are scientific institutions where was working awarded individual. Nobel Prizes in peace and literature are not related to scientific research; therefore, there is no organization to add.

## Let's dive into interesting stuff
From now on the cells are focused on finding out interesting information.

### Who won the most Nobel Prizes?
We will approach this questions from different points of view. We will look on individuals as well as organizations or countries.

In [ ]:
%%sql
/* Who won more than one Nobel Prize? */

SELECT full_name,
    COUNT(laureate_id) AS nr_of_prizes,
    sex,
    birth_date,
    death_date,
    birth_country,
    death_country
FROM nobel
GROUP BY laureate_id
HAVING COUNT(laureate_id)>1
ORDER BY nr_of_prizes DESC, birth_date DESC;

In [ ]:
%%sql
/* Record holds Red Cross, lets check in which years it was awarded */

SELECT 
    full_name,
    year,
    category,
    prize_share    
FROM nobel
WHERE full_name LIKE "%Red Cross%";

In [ ]:
%%sql
/* From which countries are comming laureates mostly? */

SELECT birth_country, COUNT(*) AS nr_prizes_birth
FROM nobel
WHERE birth_country IS NOT NULL
GROUP BY birth_country
ORDER BY nr_prizes_birth DESC, birth_country ASC
LIMIT 15;

In [ ]:
%%sql
/* How many laureates worked in other country than the country of birth? */

WITH country_difference AS(SELECT 
    CASE WHEN organization_country = birth_country THEN 'Birth and organization in the same country'
    WHEN organization_country <> birth_country THEN 'Birth and organization in different countries'
    WHEN organization_country IS NULL OR birth_country IS NULL THEN 'Birth or organization country unknown' 
    ELSE 'unexpected' END AS status 
    FROM nobel)
SELECT status,COUNT(*) AS number_of_prizes
FROM country_difference
GROUP BY status
ORDER BY number_of_prizes DESC;

In [ ]:
%%sql
/* What is the difference in number of awards by birth_country and organization_country? */

WITH organization AS (
    SELECT organization_country, COUNT(*) AS nr_prizes_org
    FROM nobel
    GROUP BY organization_country),
birth_nobel AS (
    SELECT birth_country, COUNT(*) AS nr_prizes_birth
    FROM nobel
    GROUP BY birth_country)
SELECT organization_country, nr_prizes_org, nr_prizes_birth, (nr_prizes_org-nr_prizes_birth) AS org_over_birth_nr
FROM organization AS org
JOIN birth_nobel AS birth
ON org.organization_country = birth.birth_country
ORDER BY org_over_birth_nr DESC;

In [ ]:
%%sql
/* There are many famous universities and research institutes in the world. 
How they stand in the number of Nobel Prizes awarded to their scientist? */

SELECT
    organization_name, 
    organization_country,
    COUNT(*) AS prizes_nr
FROM nobel
WHERE organization_country IS NOT NULL
GROUP BY organization_name
HAVING COUNT(*) > 9
ORDER BY prizes_nr DESC;

In [ ]:
%%sql
WITH prizes_org AS (
    SELECT COUNT(*) AS prizes_nr
    FROM nobel
    WHERE organization_country IS NOT NULL
    GROUP BY organization_name
    HAVING COUNT(*) > 9)
SELECT 
    COUNT(prizes_nr) AS nr_organizations_with_10_or_more_laureates,
    SUM(prizes_nr) AS total_nr_prizes_for_top_organizations
FROM prizes_org;

In [ ]:
%%sql
/* As I am from Czech Republic, we will look up which laureates were from Czechia or worked in Czech organization */

SELECT 
    full_name,
    year AS year_awarded,
    strftime('%Y', death_date) AS year_of_death,
    ROUND((julianday(death_date) - julianday(birth_date))/365.2422,1) AS life_length,
    category,
    motivation,
    birth_country,
    organization_country
FROM nobel
WHERE birth_country LIKE '%Cz%'
OR organization_country LIKE '%Cz%'
ORDER BY year_awarded ASC;

### Is there a day or month to be born which makes you more likely to get Nobel Prize?

In [ ]:
%%sql
/* First, lets look on birthdays. */

SELECT
strftime('%m/%d',birth_date) AS birthday,
COUNT(*) AS nr_people
FROM nobel
WHERE birthday IS NOT NULL
GROUP BY birthday
ORDER BY nr_people DESC;

In [ ]:
%%sql
/* There are no dates which would be outliers. Lets look, how are statistics by months */

SELECT
strftime('%m',birth_date) AS birthday_month,
COUNT(*) AS nr_people
FROM nobel
WHERE birthday_month IS NOT NULL
GROUP BY birthday_month
ORDER BY nr_people DESC;

In [ ]:
%%sql
/* How are statistics by day of month? */

SELECT
strftime('%d',birth_date) AS birthday_day,
COUNT(*) AS nr_people
FROM nobel
WHERE birthday_day IS NOT NULL
GROUP BY birthday_day
ORDER BY nr_people DESC;

In [ ]:
%%sql
/* Finally, how are statistics by day of week? */

SELECT
  case cast (strftime('%w', birth_date) as integer)
  WHEN 0 then 'Sunday'
  WHEN 1 then 'Monday'
  WHEN 2 then 'Tuesday'
  WHEN 3 then 'Wednesday'
  WHEN 4 then 'Thursday'
  WHEN 5 then 'Friday'
  ELSE 'Saturday' END AS birth_weekday,
COUNT(*) AS nr_people
FROM nobel
WHERE birth_date IS NOT NULL
GROUP BY birth_weekday
ORDER BY nr_people DESC;

### Summary for birthdays
We have found that 28th February was the most common birthday between all laureates. Exactly eight people was born on that day. Nevertheless, there are many other dates on which was celebrating birthdays seven or six laureates.

Statistics for months are more interesting. It is no surprise that the least laureates was born in the shortest month, February. However, the difference from other months is too large to be explained by 2-3 missing days in comparison to other months. Clearly, we would have to take in count statistics for birth rate to find out if there is born statisticaly more Nobel laureates in any month. But let's take it easy and look the numbers as they are. Most of laureates was born in June and September which have actually only 30 days. 

Further, there are quite large differences between days of month. The highest number of laureates was born on 23rd (42 laureates). One would expect that the lowest number would be born on 31st (24 laureates) but the lowest number was actually born on 17th (17 laureates). There is no reason to think that any date is better than other and the differences seems to follow normal distribution. We can assume that numbers for each date would be more similar if there would multiple times more laureates than we have today.

Finally, for days of week we have only seven options and with almost 1000 of individuals with valid birth date we would expect that numbers for each day could be very similar. Nevertheless, there is much larger amount of laureates born on Saturdays, exactly 165 which is 20 more then on the second most popular day, Tuesday. Then it seems not lucky to be born on Sunday if you want to win a Nobel Prize. On the other hand, as discussed above, the number of laureates is not huge and differences between days of week can be random.

In [ ]:
%%sql
/* As we have analyse statistics related to exact dates of birth, we can now
do the above discussed update for birth dates which has a format YYYY-00-00. 
We will substitute the 'month-day' part for first July which is almost exactly in the middle of the year.
Therefore, the new arbitrary dates will cause maximum six month error in the further calculations.
First, we can check which laureates will be edited. */

SELECT 
birth_date,
laureate_id,
strftime('%Y',birth_date) AS year_born,
full_name
FROM nobel
WHERE birth_date IS NOT NULL AND year_born IS NULL
ORDER BY birth_date;

In [ ]:
%%sql
/* Do the update of birth_date! */

UPDATE nobel
SET birth_date = substr(birth_date, 1, 4) || '-07-01'
WHERE laureate_id IN (SELECT laureate_id FROM nobel WHERE birth_date IS NOT NULL AND strftime('%Y',birth_date) IS NULL);

In [ ]:
%%sql
/* Check the result of update. There should be no record for the conditions now. */

SELECT 
birth_date,
strftime('%Y',birth_date) AS year_born,
full_name
FROM nobel
WHERE birth_date IS NOT NULL AND year_born IS NULL
ORDER BY birth_date;

In [ ]:
%%sql
/* Now we can check laureates born on first July. */

SELECT 
birth_date,
strftime('%m/%d',birth_date) AS date_born,
full_name
FROM nobel
WHERE date_born = '07/01'
ORDER BY birth_date;

### What is the average age of Nobel laureates and other age related statistics
Now, our data were cleaned in the birth_date column and we can use it to calculate age related statistics.

In [ ]:
%%sql
/* Who was the earliest and the latest born laureate so far? */
SELECT
    full_name,
    sex,
    category,
    birth_country,
    birth_date,
    year AS year_awarded,
    motivation
FROM nobel
WHERE birth_date = (SELECT MIN(birth_date) FROM nobel) 
OR birth_date = (SELECT MAX(birth_date) FROM nobel);

In [ ]:
%%sql
/* What is average age of laureates by category since year 1901? */

SELECT 
    category,
    ROUND(AVG(year - strftime('%Y', birth_date)),1) AS age_when_awarded
FROM nobel
GROUP BY category
ORDER BY age_when_awarded DESC;

In [ ]:
%%sql
/* What was average age of laureates when they were awarded and how was this number changing in time (by decades)? */

SELECT
  CAST(FLOOR(year / 10) * 10 AS TEXT) || 's' AS decade,
    ROUND(AVG(year - strftime('%Y', birth_date)),1) AS age_when_awarded
FROM nobel
GROUP BY decade;

In [ ]:
%%sql
/* Lets check, how was the average age in each category changing through the decades. */

SELECT
  CAST(FLOOR(year / 10) * 10 AS TEXT) || 's' AS decade,
    ROUND(AVG(CASE WHEN category='Physics' THEN year-strftime('%Y', birth_date) ELSE NULL END),1) AS Physics,
    ROUND(AVG(CASE WHEN category='Chemistry' THEN year-strftime('%Y', birth_date) ELSE NULL END),1) AS Chemistry,
    ROUND(AVG(CASE WHEN category='Medicine' THEN year-strftime('%Y', birth_date) ELSE NULL END),1) AS Medicine,
    ROUND(AVG(CASE WHEN category='Literature' THEN year-strftime('%Y', birth_date) ELSE NULL END),1) AS Literature,
    ROUND(AVG(CASE WHEN category='Peace' THEN year-strftime('%Y', birth_date) ELSE NULL END),1) AS Peace,
    ROUND(AVG(CASE WHEN category='Economics' THEN year-strftime('%Y', birth_date) ELSE NULL END),1) AS Economics
FROM nobel
GROUP BY decade;

In [ ]:
%%sql
/* Who was the oldest when awarded? */

SELECT 
    year - strftime('%Y', birth_date) AS age_when_awarded,
    full_name,
    year,
    category,
    motivation,
    birth_country,
    organization_country
FROM nobel
WHERE age_when_awarded IS NOT NULL
ORDER BY age_when_awarded DESC
LIMIT 5;

In [ ]:
%%sql
/* Who was the youngest when awarded? */

SELECT 
    year - strftime('%Y', birth_date) AS age_when_awarded,
    full_name,
    year,
    category,
    motivation,
    birth_country,
    organization_country
FROM nobel
WHERE age_when_awarded IS NOT NULL
ORDER BY age_when_awarded ASC
LIMIT 5;

In [ ]:
%%sql
/* Which Nobel laureates had the shortest life? */

SELECT 
    ROUND((julianday(death_date) - julianday(birth_date))/365.2422,1) AS life_length,
    full_name,
    year AS year_awarded,
    strftime('%Y', death_date) AS year_of_death,
    category,
    motivation,
    birth_country,
    organization_country
FROM nobel
WHERE life_length IS NOT NULL
ORDER BY life_length ASC
LIMIT 5;

In [ ]:
%%sql
/* Which of deceased Nobel laureates had the longest life so far? */

SELECT 
    ROUND((julianday(death_date) - julianday(birth_date))/365.2422,1) AS life_length,
    full_name,
    year AS year_awarded,
    strftime('%Y', death_date) AS year_of_death,
    category,
    motivation,
    birth_country,
    organization_country
FROM nobel
WHERE life_length IS NOT NULL
ORDER BY life_length DESC
LIMIT 5;

In [ ]:
%%sql
/* Who are the oldest currently living laureates and how old are they? */

SELECT 
    full_name,
    birth_date,
    year AS year_awarded,
    ROUND(year - strftime('%Y', birth_date),0) AS age_when_awarded,
    ROUND((julianday('now') - julianday(birth_date))/365.2422,1) AS current_age,
    category,
    motivation,
    birth_country,
    organization_country
FROM nobel
WHERE death_date IS NULL AND birth_date IS NOT NULL
ORDER BY current_age DESC
LIMIT 5;

In [ ]:
%%sql
/* Unfortunately, after quick check of nymes by google, it seems that all,
supposedly living laureates, in the previous table are actually deceased. 
This means that death dates are not updated frequently or properly in the Noble dataset
and we cannot rely that missing values means that laureate is still alive.

However, lets assume for a moment that the Nobel dataset is up to date and count 
how many living laureates are over age of 90. */

SELECT
    COUNT(*) AS living_winners_over_age_90
FROM nobel
WHERE death_date IS NULL 
AND birth_date IS NOT NULL 
AND ((julianday('now') - julianday(birth_date))/365.2422)>90;

In [ ]:
%%sql
/* As shown earlier, most people becomes laureates in a age where most of us finishing our careers.
Lets look who enjoyed to be Nobel laureate longest. 
In other words who lived (or still lives) longest after being awarded. 
Note: M.Eigen and M. Gell-Mann does not have death date in the Nobel dataset 
and they were removed after checking their death dates and calculating real value 
for years_lived_since_awarded which was too low to provide them place in this "TOP 10" */

SELECT 
    full_name,
    birth_date,
    year AS year_awarded,
    ROUND(year - strftime('%Y', birth_date),0) AS age_when_awarded,
    CASE WHEN death_date IS NULL THEN 'alive'
    WHEN death_date IS NOT NULL THEN 'deceased' END AS dead_or_alive,
    CASE WHEN death_date IS NULL THEN ROUND((julianday('now') - julianday(birth_date))/365.2422,1)
    WHEN death_date IS NOT NULL THEN ROUND((julianday(death_date) - julianday(birth_date))/365.2422,1) END AS life_length,
    CASE WHEN death_date IS NULL THEN (strftime('%Y', 'now') - year)
    WHEN death_date IS NOT NULL THEN (strftime('%Y', death_date) - year) END AS years_lived_since_awarded,
    category
FROM nobel
WHERE birth_date IS NOT NULL AND full_name NOT IN ('Manfred Eigen', 'Murray Gell-Mann')
ORDER BY years_lived_since_awarded DESC
LIMIT 10;

## Conclusions
In this notebook, we employed SQLite to analyse Nobel Prize Dataset. Data was imported from csv file and database was created. Then we checked data if any cleaning is necessary. This was done by focusing on birth and death dates and also by checking count of missing values in each column. Mostly, there was simple and clear explanations for missing values but there were some faulty birth dates which we substituted (updated) for first July. In the analysis, the main focus was on number of laureates and organizations awarded in different countries and later we calculated various statistics related to age. The effort to find the oldest currently living laureate helped us to find out that death dates in the Nobel dataset are often missing.

In summary, we can say that USA is the most successful country from the point of laureates born or working in there. Also, it seems that laureates are usually awarded in rather older age and quite a number of them lived very long lives. 